In [9]:
import requests
import xml.etree.ElementTree as ET
import csv
import time
import smtplib
import random
from email.mime.text import MIMEText


## API KEYS

In [10]:
#TOMTOM_API_KEY = 'U43oanQyySyqXVfYZAb0t2zgcdHgAVE0'

In [11]:
TOMTOM_API_KEYS = [
    "U43oanQyySyqXVfYZAb0t2zgcdHgAVE0",
    "xdmIEARrmlgFyJkuN2k0LyIkLHAwBRfn",
    "0byMrGBZACqBbHAm82Ninbh7AXOQKsfS",
    "tXbGeYgMkTU2flRpIvAaiq6fWUoJcwMh",
    "EHezDkrlBggxjoRbaQNf90F6KQUHq3Ml",
    "ZpGOoE75enqIUVCpA6iQDGyoYM6K287U",
    "ccbbacb6affa4dfdb45f796b2c369773",
    "oEynDOAUGH7GaxtB1non0xrykJni5Wl2",
    "OsNmXsu1vWunGbwvUb7wyDVgRu5RJbrN",
    "4c8XcPNXRJjfZ1Xm8UfxoffD0xXO6jyk",
    "k7sPlxsOAmHCaGfUYzeQvV88oa7kEQ91",
    "qQG7HoUihKQQi5CzsMHhCEZ9o3hvmucO",
    "A0jx4kRAuaiLmlVa1pRgyAyvEUEHLGPA",
    "WzqLqD15NjpDcVdf1IQ2djFTKKJQbqhz",
    "MytA6zsvERnG2OaHGoeiwqX3sEAky6As"
]

In [12]:
CTA_API_KEY = 'ccbbacb6affa4dfdb45f796b2c369773'

## MAP ID VALUES

In [13]:
 mapid_values = ['40260', '40380', '41700', '40680', '40850', 
                 '40160', '40040', '40730', '40460', '40330', 
                 '41400', '40350', '41160', '41220', '41120', 
                 '41320', '40820', '40960', '41020', '40910', 
                 '40740', '40740', '40310', '40670', '40220',
                 '41480', '41080', '41230', '41130', '41000',
                 '41690', '41450', '40710', '40490', '41060',
                 '40190', '40170', '40320', '40660', '40830',
                 '40210', '40120', '40810', '40590', '40530',
                 '41070', '41040', '41150', '41170', '40510',
                 '40250', '40540', '41500', '40150', '40030',
                 '40550', '40880', '40240', '40070', '40560']

In [14]:
new_mapid_values = []
with open('stops.txt', 'r', newline='', encoding='utf-8') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        stop_id = row.get('stop_id', '')
        if stop_id.isdigit() and 40000 <= int(stop_id) <= 49999:
            new_mapid_values.append(stop_id)
            
new_mapid_values

['40010',
 '40020',
 '40030',
 '40040',
 '40050',
 '40060',
 '40070',
 '40080',
 '40090',
 '40100',
 '40120',
 '40130',
 '40140',
 '40150',
 '40160',
 '40170',
 '40180',
 '40190',
 '40210',
 '40220',
 '40230',
 '40240',
 '40250',
 '40260',
 '40270',
 '40280',
 '40290',
 '40300',
 '40310',
 '40320',
 '40330',
 '40350',
 '40360',
 '40370',
 '40380',
 '40390',
 '40400',
 '40420',
 '40430',
 '40440',
 '40450',
 '40460',
 '40470',
 '40480',
 '40490',
 '40510',
 '40520',
 '40530',
 '40540',
 '40550',
 '40560',
 '40570',
 '40580',
 '40590',
 '40600',
 '40610',
 '40630',
 '40650',
 '40660',
 '40670',
 '40680',
 '40690',
 '40700',
 '40710',
 '40720',
 '40730',
 '40740',
 '40750',
 '40760',
 '40780',
 '40790',
 '40800',
 '40810',
 '40820',
 '40830',
 '40840',
 '40850',
 '40870',
 '40880',
 '40890',
 '40900',
 '40910',
 '40920',
 '40930',
 '40940',
 '40960',
 '40970',
 '40980',
 '40990',
 '41000',
 '41010',
 '41020',
 '41030',
 '41040',
 '41050',
 '41060',
 '41070',
 '41080',
 '41090',
 '41120',


## LOOP CTA TRAINS AND ABOVE-GROUND TRAFFIC SCRIPT

In [15]:
def fetch_traffic_speed(latitude, longitude, retry_keys=None):
    """
    Fetch traffic speed data using TomTom API for a given latitude and longitude.

    Parameters:
        latitude (float): Latitude coordinate.
        longitude (float): Longitude coordinate.
        retry_keys (list, optional): List of TomTom API keys to retry in case of errors.

    Returns:
        tuple: A tuple containing free flow speed and current speed in mph.
    """
        
    traffic_url = 'https://api.tomtom.com/traffic/services/4/flowSegmentData/relative/10/json'
    
    api_keys = retry_keys or TOMTOM_API_KEYS
    remaining_keys = list(api_keys)
    
    retry_seconds = 2  # Default retry time in seconds
    
    while remaining_keys:
        current_api_key = remaining_keys.pop(0)
        
        traffic_params = {
            'point': f'{latitude},{longitude}',
            'unit': 'mph',
            'key': current_api_key
        }
        
        try:
            traffic_response = requests.get(traffic_url, params=traffic_params)
            
            if traffic_response.status_code == 200:
                try:
                    traffic_data = traffic_response.json()
                    if "flowSegmentData" in traffic_data:
                        flow_segment_data = traffic_data["flowSegmentData"]
                        free_flow_speed_mph = flow_segment_data.get("freeFlowSpeed", "N/A")
                        current_speed_mph = flow_segment_data.get("currentSpeed", "N/A")
                        return free_flow_speed_mph, current_speed_mph
                    else:
                        print("TomTom API response does not contain flowSegmentData.")
                except json.JSONDecodeError as e:
                    print("Error decoding JSON response:", e)
            elif traffic_response.status_code == 403:
                print(f"TomTom API request error - status code 403 (Forbidden). Switching to the next available key...")
                if remaining_keys:
                    print(f"Retrying with the next key in {retry_seconds} seconds...")
                    time.sleep(retry_seconds)  # Wait for the specified seconds before retrying
                    continue  # Retry with the next key
                else:
                    print("Exhausted all available API keys.")
            else:
                print("TomTom API request error - status code:", traffic_response.status_code)
            
        except requests.exceptions.RequestException as e:
            print("Request error:", e)
            if isinstance(e, requests.exceptions.ConnectionError) and "forcibly closed by the remote host" in str(e):
                if remaining_keys:
                    print("Retrying with the next key...")
                    continue  # Retry with the next key
                else:
                    print("Exhausted all available API keys.")
            break  # Exit loop for other types of errors
    
    print("Exhausted all available API keys.")
    return None, None

def fetch_CTA_and_traffic():
    """
    Fetch CTA arrival data and combine it with traffic speed data, then save to CSV file.

    This function continuously fetches CTA arrival data, fetches traffic speed using TomTom API,
    combines the data, and saves it to a CSV file every 5 minutes.
    """
        
    cta_url = 'https://lapi.transitchicago.com/api/1.0/ttarrivals.aspx'

    while True:
        with open('combined_data.csv', 'a', newline = '', encoding = 'utf-8') as csv_file:
            csv_writer = csv.writer(csv_file)
            
            # Shuffle the new_mapid_values list
            random.shuffle(new_mapid_values)
            
            for mapid in new_mapid_values:
                cta_params = {
                    'mapid': mapid,
                    'max': '7', 
                    'key': CTA_API_KEY
                }
                
                try:
                    cta_response = requests.get(cta_url, params=cta_params)
                    cta_response.raise_for_status()  # Raise an exception for HTTP errors

                    if cta_response.status_code == 200:
                        root = ET.fromstring(cta_response.text)
                        eta_elements = root.findall(".//eta")

                        for eta in eta_elements:
                            route = eta.find("rt").text
                            prd_time = eta.find("prdt").text
                            arr_time = eta.find("arrT").text
                            sch_bool = eta.find("isSch").text
                            flt_bool = eta.find("isFlt").text
                            dly_bool = eta.find("isDly").text
                            station_name = eta.find("staNm").text
                            dest_name = eta.find("destNm").text

                            latitude_element = eta.find("lat")
                            longitude_element = eta.find("lon")

                            if latitude_element is not None and longitude_element is not None:
                                latitude = latitude_element.text
                                longitude = longitude_element.text

                                if latitude and longitude:
                                    free_flow_speed, current_speed = fetch_traffic_speed(float(latitude), float(longitude))

                                    if free_flow_speed is not None:
                                        csv_writer.writerow([mapid, route, prd_time, arr_time, sch_bool, flt_bool, dly_bool, station_name, dest_name, latitude, longitude, free_flow_speed, current_speed])
                                else:
                                    print("Invalid latitude or longitude in CTA API response.")
                            else:
                                print("Missing latitude or longitude in CTA API response.")
                    
                except requests.exceptions.RequestException as e:
                    print("Request error:", e)
                    if isinstance(e, requests.exceptions.ConnectionError) and "forcibly closed by the remote host" in str(e):
                        print("Waiting for 30 seconds before retrying...")
                        time.sleep(30)  # Wait for 30 seconds before retrying
                        continue  # Retry with the next key
                    else:
                        break  # Exit loop for other types of errors
        
        time.sleep(5 * 60)  # x minutes in seconds

fetch_CTA_and_traffic()

TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
Invalid latitude or longitude in CTA API response.
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
Invalid latitude or longitude in CTA API response.
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
Invalid latitude or longitude in CTA API response.
TomTom API request 

Invalid latitude or longitude in CTA API response.
Invalid latitude or longitude in CTA API response.
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
Invalid latitude or longitude in CTA API response.
Invalid latitude or longitude in CTA API response.
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
Invalid latitude or longitude in CTA API response.
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden).

TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
Invalid latitude or longitude in CTA API response.
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key

TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
Invalid latitude or longitude in CTA API response.
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key

TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
Invalid latitude or longitude in CTA API response.
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
Invalid latitude or longitude in CTA API response.
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to t

...

Invalid latitude or longitude in CTA API response.
Invalid latitude or longitude in CTA API response.
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
Invalid latitude or longitude in CTA API response.
Invalid latitude or longitude in CTA API response.
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying wi

Invalid latitude or longitude in CTA API response.
Invalid latitude or longitude in CTA API response.
Invalid latitude or longitude in CTA API response.
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
Invalid latitude or longitude in CTA API response.
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
Invalid latitude or longitude in CTA API response.
Invalid latitude or longitude in CTA API response.
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomT

TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status 

TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
Invalid latitude or longitude in CTA API response.
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
TomTom API request error - status code 403 (Forbidden). Switching to the next available key...
Retrying with the next key in 2 seconds...
Invalid latitude or longitude in CTA API response.
TomTom API request error - status code 403 (Forbidden). Switching to t

KeyboardInterrupt: 